## Fine-Tuning GPT Models - A Python SDK Experience

Learn how to fine-tune the <code>gpt-35-turbo-0613</code> using Python Programming Language - An SDK / Code Experience. This notebook is based on the MS Learn tutorial [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=python%2Cbash).

He Zhang, Feb. 2024

### Prerequisites

* Learn the [what, why, and when to use fine-tuning.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/fine-tuning-considerations)
* An Azure subscription.
* Access to Azure OpenAI Service.
* An Azure OpenAI resource created in the supported fine-tuning region (e.g. Sweden Central).
* Prepare Training and Validation datasets:
  * at least 50 high-quality samples (preferably 1,000s) are required.
  * must be formatted in the JSON Lines (JSONL) document with UTF-8 encoding.
  * for this test notebook, we use only 10 samples for the demo purpose. 
* Python version at least: <code>3.7.1</code>
* Python libraries: <code>json, requests, os, tiktoken, time, openai, notebook</code>
* The OpenAI Python library version at least: <code>0.28.1</code>

In [1]:
import datetime
import openai
import os
import json
import numpy as np
import requests
import sys
import time
import tiktoken

from dotenv import load_dotenv
from IPython.display import clear_output

print("openai version:", openai.__version__)

openai version: 0.28.1


In [2]:
load_dotenv("azure.env")

openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-09-15-preview"
print("AOAI Base:", openai.api_base)

AOAI Base: https://explore-openai-sweden-central.openai.azure.com/


In [3]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 14-Dec-2023 02:52:26


In [5]:
print("Open AI version:", openai.__version__)

Open AI version: 0.28.1


## Testing

In [6]:
model = "gpt-35-turbo-16k"

In [7]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[{"role": "user", "content": "Hello, my name is He."}],
    max_tokens=100,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print("Response:\n\n", type(response), response)

Response:

 <class 'openai.openai_object.OpenAIObject'> {
  "id": "chatcmpl-8VWALEx0fRwmiCrPpxhhz8LYtZMc8",
  "object": "chat.completion",
  "created": 1702522505,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello, He! How can I assist you today?"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
   

In [8]:
print("Answer type:\n\n", type(response.choices[0].message))

Answer type:

 <class 'openai.openai_object.OpenAIObject'>


In [9]:
print("Answer:\n\n", response.choices[0].message)

Answer:

 {
  "role": "assistant",
  "content": "Hello, He! How can I assist you today?"
}


## Training and validation Jsonl files

In [10]:
training_file_name = "training_set.jsonl"
validation_file_name = "validation_set.jsonl"

In [11]:
!ls $training_file_name -lh

-rwxrwxrwx 1 root root 2.7K Dec 11 06:42 training_set.jsonl


In [12]:
with open(training_file_name, "r") as f:
    print(f.read())

{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who discovered Antarctica?"}, {"role": "assistant", "content": "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What is the biggest ocean?"}, {"role": "assistant", "content": "The Pacific Ocean. It's not like it's a small pond or anything."}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What is the largest planet?"}, {"role": "assistant", "content": "It's called Jupiter, you may have heard of it...or not."}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who painted the Mona Lisa?"}, {"role": "a

In [13]:
!ls $validation_file_name -lh

-rwxrwxrwx 1 root root 2.6K Dec 11 06:42 validation_set.jsonl


In [14]:
with open(validation_file_name, "r") as f:
    print(f.read())

{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of Australia?"}, {"role": "assistant", "content": "It's Canberra, not Sydney. Shocking, I know!"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'The Great Gatsby'?"}, {"role": "assistant", "content": "Some guy named F. Scott Fitzgerald. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the deepest part of the ocean?"}, {"role": "assistant", "content": "Mariana Trench, makes your swimming pool seem a bit shallow, doesn't it?"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who composed the 'Moonlight Sonata'?"}, {"role": "assistant", "content": "Just a little

## Fine tuning

In [15]:
# Load the training set
with open(training_file_name, "r", encoding="utf-8") as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 10
First example in training set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': 'Who discovered Antarctica?'}
{'role': 'assistant', 'content': "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}


In [16]:
# Load the validation set
with open(validation_file_name, "r", encoding="utf-8") as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)


Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the capital of Australia?"}
{'role': 'assistant', 'content': "It's Canberra, not Sydney. Shocking, I know!"}


## Tokens information

In [17]:
encoding = tiktoken.get_encoding("cl100k_base")
encoding

<Encoding 'cl100k_base'>

In [18]:
encoding.encode("hello there, baby He")

[15339, 1070, 11, 8945, 1283]

In [19]:
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """
    Number of tokens from messages
    """
    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name

    num_tokens += 3

    return num_tokens

In [20]:
def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))

    return num_tokens

In [21]:
def print_distribution(values, name):
    """
    Stats
    """
    print(f"Distribution of {name}:")
    print(f"   min / max: {min(values)}, {max(values)}")
    print(f"   mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"   p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")
    print()

In [29]:
files = [training_file_name, validation_file_name]

for file in files:
    print()
    print("-" * 100)
    print(f"Processing file: {file}")
    print("-" * 100)

    with open(file, "r", encoding="utf-8") as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")


----------------------------------------------------------------------------------------------------
Processing file: training_set.jsonl
----------------------------------------------------------------------------------------------------
Distribution of total tokens:
   min / max: 47, 62
   mean / median: 52.1, 50.5
   p5 / p95: 47.9, 57.5

Distribution of assistant tokens:
   min / max: 13, 30
   mean / median: 17.6, 15.5
   p5 / p95: 13.0, 21.9


----------------------------------------------------------------------------------------------------
Processing file: validation_set.jsonl
----------------------------------------------------------------------------------------------------
Distribution of total tokens:
   min / max: 43, 65
   mean / median: 51.4, 49.0
   p5 / p95: 45.7, 56.9

Distribution of assistant tokens:
   min / max: 8, 29
   mean / median: 15.9, 13.5
   p5 / p95: 11.6, 20.9



In [30]:
total_tokens, assistant_tokens

([48, 49, 56, 49, 65, 46, 56, 54, 48, 43],
 [13, 13, 19, 13, 29, 12, 18, 20, 14, 8])

## Running the fine tuning model job

In [31]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=training_file_name,
)
training_file_id = training_response["id"]


validation_response = openai.File.create(
    file=open(validation_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=validation_file_name,
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-96b976a2e932436b901b1bfcb801cafe
Validation file ID: file-cd45f4da179b4f07b5c55dc23d6d00b9


# Fine tuning model

**babbage-002** and **davinci-002** are not trained to follow instructions. Querying these base models should only be done as a point of reference to a fine-tuned version to evaluate the progress of your training.

**gpt-35-turbo-0613** - fine-tuning of this model is limited to a subset of regions, and is not available in every region the base model is available.

In [35]:
ftmodel = "gpt-35-turbo-0613" # note by He Zhang: has to be exactly this name, not the deployment name.
#ftmodel = "gpt-35-turbo-16k" # this does not work

In [36]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=ftmodel,
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)

Job ID: ftjob-46e808787e8e47d38bd5a85741b394cc
Status: pending
{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "pending",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-96b976a2e932436b901b1bfcb801cafe",
  "validation_file": "file-cd45f4da179b4f07b5c55dc23d6d00b9",
  "id": "ftjob-46e808787e8e47d38bd5a85741b394cc",
  "created_at": 1702523504,
  "updated_at": 1702523504,
  "object": "fine_tuning.job"
}


In [48]:
def check_training_status():
    """
    Check fine tuning process
    """
    response = openai.FineTuningJob.retrieve(job_id)

    if response["status"] == "pending":
        print(f"Fine tuning of {ftmodel} is PENDING - Please wait")

    elif response["status"] == "running":
        print("\033[1;31;34m")
        print(f"Fine tuning of {ftmodel} is RUNNING - Please wait")

    elif response["status"] == "succeeded":
        print("\033[1;31;32m")
        print(f"Fine tuning of {ftmodel} is DONE!")

    else:
        print(response)

    print("\nJob ID:", job_id)
    print("Created:", datetime.datetime.utcfromtimestamp(response["created_at"]))
    print("Updated:", datetime.datetime.utcfromtimestamp(response["updated_at"]))

    print("\033[0m")
    print(response)

In [38]:
check_training_status()


Fine tuning of gpt-35-turbo-0613 is RUNNING - Please wait

Job ID: ftjob-46e808787e8e47d38bd5a85741b394cc
Created: 2023-12-14 03:11:44
Updated: 2023-12-14 03:13:17

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "running",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-96b976a2e932436b901b1bfcb801cafe",
  "validation_file": "file-cd45f4da179b4f07b5c55dc23d6d00b9",
  "id": "ftjob-46e808787e8e47d38bd5a85741b394cc",
  "created_at": 1702523504,
  "updated_at": 1702523597,
  "object": "fine_tuning.job"
}


In [49]:
check_training_status()


Fine tuning of gpt-35-turbo-0613 is DONE!

Job ID: ftjob-46e808787e8e47d38bd5a85741b394cc
Created: 2023-12-14 03:11:44
Updated: 2023-12-14 03:40:52

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc",
  "training_file": "file-96b976a2e932436b901b1bfcb801cafe",
  "validation_file": "file-cd45f4da179b4f07b5c55dc23d6d00b9",
  "result_files": [
    "file-36cdaed7bcec4702bbfdbff099b82914"
  ],
  "finished_at": 1702525252,
  "trained_tokens": 1042,
  "id": "ftjob-46e808787e8e47d38bd5a85741b394cc",
  "created_at": 1702523504,
  "updated_at": 1702525252,
  "object": "fine_tuning.job"
}


> https://oai.azure.com/portal

In [50]:
openai.FineTuningJob.list()

<OpenAIObject list at 0x7f3c40b24f40> JSON: {
  "has_more": false,
  "data": [
    {
      "hyperparameters": {
        "n_epochs": 2
      },
      "status": "succeeded",
      "model": "gpt-35-turbo-0613",
      "fine_tuned_model": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc",
      "training_file": "file-96b976a2e932436b901b1bfcb801cafe",
      "validation_file": "file-cd45f4da179b4f07b5c55dc23d6d00b9",
      "result_files": [
        "file-36cdaed7bcec4702bbfdbff099b82914"
      ],
      "finished_at": 1702525252,
      "trained_tokens": 1042,
      "id": "ftjob-46e808787e8e47d38bd5a85741b394cc",
      "created_at": 1702523504,
      "updated_at": 1702525252,
      "object": "fine_tuning.job"
    },
    {
      "hyperparameters": {
        "n_epochs": 2
      },
      "suffix": "gpt-35-turbo-ft-aoai-dashboard",
      "status": "succeeded",
      "model": "gpt-35-turbo-0613",
      "fine_tuned_model": "gpt-35-turbo-0613.ft-88836731c3a1455eb5c153ab06ce6ea0-gpt-35-turbo-ft-

<img src="capture_hezhang.png">


## Deploying the model

- token:	There are multiple ways to generate an authorization token. The easiest method for initial testing is to launch the Cloud Shell from the Azure portal. Then run az account get-access-token. You can use this token as your temporary authorization token for API testing. We recommend storing this in a new environment variable
- subscription:	The subscription ID for the associated Azure OpenAI resource
- resource_group:	The resource group name for your Azure OpenAI resource
- resource_name:	The Azure OpenAI resource name
- model_deployment_name:	The custom name for your new fine-tuned model deployment. This is the name that will be referenced in your code when making chat completion calls.

In [51]:
# Retrieve fine_tuned_model name
response = openai.FineTuningJob.retrieve(job_id)

print(response)
fine_tuned_model = response["fine_tuned_model"]

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc",
  "training_file": "file-96b976a2e932436b901b1bfcb801cafe",
  "validation_file": "file-cd45f4da179b4f07b5c55dc23d6d00b9",
  "result_files": [
    "file-36cdaed7bcec4702bbfdbff099b82914"
  ],
  "finished_at": 1702525252,
  "trained_tokens": 1042,
  "id": "ftjob-46e808787e8e47d38bd5a85741b394cc",
  "created_at": 1702523504,
  "updated_at": 1702525252,
  "object": "fine_tuning.job"
}


In [54]:
#token = os.getenv("TOKEN")
token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSIsImtpZCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuY29yZS53aW5kb3dzLm5ldC8iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8xNmIzYzAxMy1kMzAwLTQ2OGQtYWM2NC03ZWRhMDgyMGI2ZDMvIiwiaWF0IjoxNzAyNTI4MDg3LCJuYmYiOjE3MDI1MjgwODcsImV4cCI6MTcwMjUzMjk0OCwiYWNyIjoiMSIsImFpbyI6IkFXUUFtLzhWQUFBQTlZeTBTalQ4N0dZUjh0eHRPNGYxanVFVVl3aFdZMGg2a0E2MW41eGYxbEg0bGlRTXJoT3BLbk1MaDFkdFZOTnR4N1E1czZZTzFmNmV4RTl1YmhhNXU4TmwwVmp1elZiSTg4NTBZd1BGMHAxQ0IzWkV2SE0zR1RVV0ZxYlJMbzl1IiwiYWx0c2VjaWQiOiI1OjoxMDAzMjAwMTdDQjUyQTM2IiwiYW1yIjpbInJzYSIsIm1mYSJdLCJhcHBpZCI6ImI2NzdjMjkwLWNmNGItNGE4ZS1hNjBlLTkxYmE2NTBhNGFiZSIsImFwcGlkYWNyIjoiMCIsImRldmljZWlkIjoiNmYwMjdjMDMtYjBiOC00MWMwLTkyNGQtN2NjZjRhOWYxYWVjIiwiZW1haWwiOiJ6aGFuZ2hlQG1pY3Jvc29mdC5jb20iLCJmYW1pbHlfbmFtZSI6IlpoYW5nIiwiZ2l2ZW5fbmFtZSI6IkhlIiwiZ3JvdXBzIjpbImIxMzA0MDIyLTA4ZTYtNDQ3ZC1iMDk0LTE1MzcwNTk3YzZiNiIsIjA5M2I4YTIyLTg0OWYtNGUyYi1iM2M3LTc1YmZlMzA1MDU0NiIsImQzNGM0ZWJlLTQ5ODQtNDkwMy1hNjRkLThjMjAyODNkNTE2YiIsImUzMDk2ZGY3LWI2NWMtNGUzMi1hYjFhLTdhMzVkYzY4NGYwYSJdLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC83MmY5ODhiZi04NmYxLTQxYWYtOTFhYi0yZDdjZDAxMWRiNDcvIiwiaWR0eXAiOiJ1c2VyIiwiaXBhZGRyIjoiMjQwNDpmODAxOjkwMDA6MWE6ZWZlYTo6MzRmIiwibmFtZSI6IkhlIFpoYW5nIiwib2lkIjoiMGY2NTM4NTItOGVlMS00YTAzLWI2NTMtMTUzYzdjYjQ5MDEzIiwicHVpZCI6IjEwMDMyMDAyNjQ1QTExQjkiLCJyaCI6IjAuQVVZQUU4Q3pGZ0RUalVhc1pIN2FDQ0MyMDBaSWYza0F1dGRQdWtQYXdmajJNQk84QU1RLiIsInNjcCI6InVzZXJfaW1wZXJzb25hdGlvbiIsInN1YiI6Ilp5a2ZTMlEwQlJDakJtU2VwUU9pdElJRnFMQUJ6SDQtOFBOWVpnOFVFd1UiLCJ0aWQiOiIxNmIzYzAxMy1kMzAwLTQ2OGQtYWM2NC03ZWRhMDgyMGI2ZDMiLCJ1bmlxdWVfbmFtZSI6InpoYW5naGVAbWljcm9zb2Z0LmNvbSIsInV0aSI6ImJQZTROVnVyUkV1eDROLW52OFVLQVEiLCJ2ZXIiOiIxLjAiLCJ3aWRzIjpbImI3OWZiZjRkLTNlZjktNDY4OS04MTQzLTc2YjE5NGU4NTUwOSJdLCJ4bXNfY2FlIjoiMSIsInhtc190Y2R0IjoxNjQ1MTM3MjI4fQ.PmGW5Iv9LuUVg_a0BhWoOGTxEpM-nVNTEr46KcN5bvQQi19cjtQjP3-dHDzYcaZcNSZUnuRHrltq_hW5gFxPTzHKiYQtq2_VtcIGdjaUMJSuheBVz1PYQPtPowFsmQt7uMoStHnSQFOGkgjBNplWPt1hgCOx5NE3f8qU2mrgye4dJgE3TvgEWfa2JrPEjgCH_-TioFqhvUsdV0F50oxLusSuqbi2eK1MDZI8pqmRYTTAWxLHQSP5RUUNGT2AB-0ASJxhXOSs9kYqN2NYnkuSOEqNM57R53Z8nduWAoWeRHcTX_TScjfpyn6Z2nKBQnOq2y4F2C9YU7lWpBqrKnVH0Q"
subscription = os.getenv("SUBSCRIPTION")
resource_group = os.getenv("RESOURCE_GROUP")
resource_name = os.getenv("RESOURCE_NAME")

model_deployment_name = "gpt-35-turbo-ft-aoai-sdk"

In [56]:
deploy_params = {"api-version": "2023-05-01"} # "2023-07-01-preview"
deploy_headers = {
    "Authorization": "Bearer {}".format(token),
    "Content-Type": "application/json",
}

deploy_data = {
    "sku": {"name": "standard", "capacity": 10},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc",  # retrieve this value from the previous call
            "version": "1",
        }
    },
}

deploy_data = json.dumps(deploy_data)
request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"
print(request_url)

https://management.azure.com/subscriptions/02243ba5-b777-47c6-9ecf-830b204b7593/resourceGroups/OpenAI/providers/Microsoft.CognitiveServices/accounts/explore-openai-sweden-central/deployments/gpt-35-turbo-ft-aoai-sdk


In [57]:
deploy_data

'{"sku": {"name": "standard", "capacity": 10}, "properties": {"model": {"format": "OpenAI", "name": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc", "version": "1"}}}'

In [58]:
print("Creating a new deployment...")
resp = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)
print(resp)

Creating a new deployment...
<Response [201]>


In [64]:
resp.json()

{'id': '/subscriptions/02243ba5-b777-47c6-9ecf-830b204b7593/resourceGroups/OpenAI/providers/Microsoft.CognitiveServices/accounts/explore-openai-sweden-central/deployments/gpt-35-turbo-ft-aoai-sdk',
 'type': 'Microsoft.CognitiveServices/accounts/deployments',
 'name': 'gpt-35-turbo-ft-aoai-sdk',
 'sku': {'name': 'standard', 'capacity': 10},
 'properties': {'model': {'format': 'OpenAI',
   'name': 'gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc',
   'version': '1'},
  'versionUpgradeOption': 'NoAutoUpgrade',
  'capabilities': {'chatCompletion': 'true'},
  'provisioningState': 'Creating',
  'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 10},
   {'key': 'token', 'renewalPeriod': 60, 'count': 10000}]},
 'systemData': {'createdBy': 'zhanghe@microsoft.com',
  'createdByType': 'User',
  'createdAt': '2023-12-14T04:43:04.3841227Z',
  'lastModifiedBy': 'zhanghe@microsoft.com',
  'lastModifiedByType': 'User',
  'lastModifiedAt': '2023-12-14T04:43:04.3841227Z'},
 'etag': '"

In [59]:
print(resp.reason)

Created


In [60]:
mymodel = resp.json()["name"]
mymodel

'gpt-35-turbo-ft-aoai-sdk'

In [65]:
resp.json()["systemData"]["createdAt"]

'2023-12-14T04:43:04.3841227Z'

In [66]:
resp.json()["properties"]["model"]

{'format': 'OpenAI',
 'name': 'gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc',
 'version': '1'}

In [67]:
status = resp.json()["properties"]["provisioningState"]

print(f"Model deployment status of model '{mymodel}': {status}")

Model deployment status of model 'gpt-35-turbo-ft-aoai-sdk': Creating


In [71]:
resp = requests.put(
    request_url, params=deploy_params, headers=deploy_headers, data=deploy_data
)

status = resp.json()["properties"]["provisioningState"]
print(f"Model deployment status of model '{mymodel}': {status}")

Model deployment status of model 'gpt-35-turbo-ft-aoai-sdk': Succeeded


## Testing the model

In [72]:
openai.api_version = "2023-05-15"

In [80]:
response = openai.ChatCompletion.create(
    engine=mymodel, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "My name is He."},
        {"role": "assistant", "content": "Hey He, how can I help you?"},
        {"role": "user", "content": "1+1=?"},
        {"role": "assistant", "content": "2"},
        {"role": "assistant", "content": "ok, my name again?"}
    ])

In [81]:
print(response)

{
  "id": "chatcmpl-8VY9tzOYbJIWNk74YbetfTeZY515P",
  "object": "chat.completion",
  "created": 1702530165,
  "model": "gpt-35-turbo-0613.ft-46e808787e8e47d38bd5a85741b394cc",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Your name is He."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 58,
    "completion_tokens": 5,
    "total_tokens": 63
  }
}


In [82]:
print("Answer type:\n\n", type(response.choices[0].message))

Answer type:

 <class 'openai.openai_object.OpenAIObject'>


In [83]:
print(response["choices"][0]["message"]["content"])

Your name is He.
